# Verify Claims

In [2]:
import csv, json, re
from collections import defaultdict
from pathlib import Path
ROOT=Path('.')
RESP=ROOT/'results'/'responses.csv'
GT=ROOT/'analysis'/'ground_truth.json'
OUT=ROOT/'analysis'/'fabrication_report.json'
if not RESP.exists(): raise SystemExit('No responses.csv — log outputs in collect_responses.ipynb')
if not GT.exists(): raise SystemExit('Missing analysis/ground_truth.json')
gt=json.loads(GT.read_text()); games=gt.get('games_played'); p1=(gt.get('period_goals_snapshot') or {}).get('P1_total'); p4=(gt.get('period_goals_snapshot') or {}).get('P4_total')
num=re.compile(r'\b(\d{1,3})\b')
contr, total=defaultdict(int), defaultdict(int)
with RESP.open(encoding='utf-8') as f:
    for row in csv.DictReader(f):
        h=row.get('hypothesis_id',''); txt=row.get('response_text',''); total[h]+=1; low=txt.lower()
        if 'game' in low and games is not None:
            for m in num.findall(txt):
                n=int(m)
                if 5<=n<=40 and n!=games: contr[h]+=1; break
        if (('first period' in low or '1st period' in low or 'p1' in low) and ('fourth period' in low or '4th period' in low or 'p4' in low)):
            if p1 is not None and p4 is not None and p4 < p1 and ('higher in fourth' in low or 'improved in fourth' in low or 'more goals in fourth' in low): contr[h]+=1
rate={h:(contr[h]/total[h] if total[h] else 0.0) for h in total}
(OUT).write_text(json.dumps({'fabrication_rate_by_hypothesis':rate,'contradictions_by_hypothesis':dict(contr),'total_responses_by_hypothesis':dict(total)}, indent=2))
print('Wrote', OUT)


Wrote analysis\fabrication_report.json
